In [ ]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np 
import os
import cloudvolume
import utils

In [ ]:
client = CAVEclient()
dataset = 'fanc_production_mar2021'
client = CAVEclient(dataset)

In [ ]:
pmns = pd.read_csv('~/Code/connectomics/connectomics/annotations_pMNs/ng_annotation_points.csv')

In [ ]:
print(pmns.shape)
pmns.head()

In [ ]:
pt_ints = []
for i in pmns['Coordinate 1'].tolist():
    #print(i)
    i = i.split('(')[1]
    i = i.split(')')[0].split(',')
    i = [j for j in i if j != '']
    
    pt_ints.append(list(map(int, i)))

print(pt_ints[0])
type(pt_ints[0])
pmns['pt_position'] = pt_ints
pmns.head()

In [ ]:
pmns['receipient_0'] = np.nan
pmns['receipient_1'] = np.nan
pmns['receipient_2'] = np.nan
pmns['classification_system'] = np.nan
pmns['cell_type'] = pmns.Description.fillna('unidentified')
pmns['NB_name'] = np.nan
pmns['NT']= np.nan

pmns_ann_table = pmns[['pt_position','receipient_0','receipient_1','receipient_2','classification_system','cell_type','NB_name','NT']].copy()
pmns_ann_table.head()


# For the purposes of checking, get segIDs for points and create a csv file

In [ ]:
# Find segIDs from points
cv = cloudvolume.CloudVolume(client.info.get_datastack_info()['segmentation_source'], use_https=True)
segIDs = utils.segIDs_from_pts_service(pmns_ann_table.pt_position,cv)

In [ ]:
pmns_ann_table['pt_root_id'] = segIDs


In [ ]:

pmns_ann_table = pmns_ann_table[['pt_root_id','pt_position','receipient_0','receipient_1','receipient_2','classification_system','cell_type','NB_name','NT']]
utils.save_df_csv(pmns_ann_table,'ng_annotation_points_for_import')

In [ ]:
pmns_ann_table